In [5]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, VGG19, InceptionV3
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

In [6]:
# Load CIFAR10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [7]:
# Choose a pre-trained model (e.g., VGG19)
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
# Add custom top layers for CIFAR10
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(512),
    layers.LeakyReLU(alpha=0.1),  # Leaky ReLU activation
    layers.Dropout(0.5),
    layers.Dense(256),
    layers.LeakyReLU(alpha=0.1),
    layers.Dropout(0.5),
    layers.Dense(128),
    layers.LeakyReLU(alpha=0.1),
    layers.Dense(10, activation='softmax')  # Output for CIFAR10
])

# Freeze base model layers for initial evaluation
for layer in base_model.layers:
    layer.trainable = False

/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [9]:
# Compile and evaluate without fine-tuning
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print("Evaluating with pre-trained weights:")
model.evaluate(x_test, y_test, verbose=2)

Evaluating with pre-trained weights:
313/313 - 7s - 23ms/step - accuracy: 0.1069 - loss: 2.3331


[2.333101272583008, 0.10689999908208847]

In [10]:
# Fine-tune the model
for layer in base_model.layers:
    layer.trainable = True  # Unfreeze for fine-tuning

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
# Train the model
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32)

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 90s 50ms/step - accuracy: 0.3255 - loss: 1.8550 - val_accuracy: 0.6974 - val_loss: 0.8933
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 129s 46ms/step - accuracy: 0.6920 - loss: 0.9477 - val_accuracy: 0.7490 - val_loss: 0.7537
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 82s 46ms/step - accuracy: 0.7631 - loss: 0.7400 - val_accuracy: 0.7892 - val_loss: 0.6448
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 71s 46ms/step - accuracy: 0.8041 - loss: 0.6121 - val_accuracy: 0.7916 - val_loss: 0.6434
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 85s 47ms/step - accuracy: 0.8360 - loss: 0.5237 - val_accuracy: 0.8218 - val_loss: 0.5388
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 79s 46ms/step - accuracy: 0.8613 - loss: 0.4386 - val_accuracy: 0.8259 - val_loss: 0.5341
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 74s 47ms/step - accuracy: 0.8784 - loss: 0.3809 - val_accuracy: 0.8283 - val_loss: 0.5545
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 80s 46ms/step - accuracy: 0.8935 

In [12]:
# Evaluate the fine-tuned model
print("Evaluating after fine-tuning:")
model.evaluate(x_test, y_test, verbose=2)

Evaluating after fine-tuning:
313/313 - 3s - 8ms/step - accuracy: 0.8380 - loss: 0.5609


[0.5608878135681152, 0.8379999995231628]